<a href="https://colab.research.google.com/github/akhiltabjula/ComputationalMethodsForInformationSystems/blob/master/INFO5731_Assignment_Three.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **INFO5731 Assignment Three**


In this assignment, you are required to conduct information extraction, semantic analysis based on the dataset you collected from assignment two. You may use scipy and numpy package in this assignment.

# **Question 1: Understand N-gram**


(45 points). Write a python program to conduct N-gram analysis based on the dataset in your assignment two:

(1) Count the frequency of all the N-grams (N=3).

(2) Calculate the probabilities for all the bigrams in the dataset by using the fomular count(w2 w1) / count(w2). For example, count(really like) / count(really) = 1 / 3 = 0.33.

(3) Extract all the noun phrases and calculate the relative probabilities of each review in terms of other reviews (abstracts, or tweets) by using the fomular frequency (noun phrase) / max frequency (noun phrase) on the whole dataset. Print out the result in a table with column name the all the noun phrases and row name as all the 100 reviews (abstracts, or tweets).

In [ ]:
# Write your code here

url = "https://www.imdb.com/title/tt7286456/reviews?sort=helpfulnessScore&dir=desc&ratingFilter=10"

In [ ]:
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [39.3 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Get:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release [564 B]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release.gpg [833 B]
Hit:10 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:11 http://security.ubuntu.com/ubuntu bionic-security InRelease

In [ ]:
import pandas as pd
from selenium import webdriver
import time

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument('-headless')
options.add_argument('-no-sandbox')
options.add_argument('-disable-dev-shm-usage')
chrome_driver = webdriver.Chrome('chromedriver',options=options)
xPath = '//*[@id ="load-more-trigger"]'

In [ ]:
chrome_driver.get(url)
for i in range(0,3):
  on_click = chrome_driver.find_element_by_xpath(xPath).click()
  time.sleep(2)

In [ ]:
texts = list()
txt = chrome_driver.find_elements_by_class_name("content")
for i in txt:
  texts.append(i.find_element_by_css_selector(".text.show-more__control").text)
texts1 = list()
txt1 = chrome_driver.find_elements_by_class_name("display-name-date")
for j in txt1:
  texts1.append(j.find_element_by_css_selector("a").text)
output = pd.concat([pd.DataFrame(texts1,columns=['Names']),pd.DataFrame(texts,columns=['review'])],axis=1)
output.head()

,Names,review
0,MihaVrhunc,"Every once in a while a movie comes, that trul..."
1,lesterarnoldpinto,This is a movie that only those who have felt ...
2,Aman_Goyal,"Truly a masterpiece, The Best Hollywood film o..."
3,logical_guy,Joaquin Phoenix gives a tour de force performa...
4,kdagoulis26,Most of the time movies are anticipated like t...


In [ ]:
def merge_two_dataframes(ip1, ip2):
  return pd.concat([ip1,ip2],axis=1)

In [ ]:
import string
def remove_spl_characters(ip):
  res = list()
  for val in ip:
    if val not in string.punctuation and (val.isalnum() or val == " "):
      res.append(val)
  return ''.join(res)


In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
words_data = set(stopwords.words('english'))

In [ ]:
def remove_stop_words(ip):
  res = list()
  for val in ip:
    if val not in words_data:
      res.append(val)
  return ' '.join(res)

In [ ]:
''' Frequent words removal  '''
def remove_frequent_words(ip):
  res = list()
  for val in ip:
    if val not in frequent_words:
      res.append(val)
  return ' '.join(res)

In [ ]:
def remove_frequent_words(ip):
  res = list()
  for val in ip:
    if val not in rare_words:
      res.append(val)
  return ' '.join(res)

In [ ]:
lower_case_df = merge_two_dataframes(output, output['review'].apply(lambda a: a.lower()).to_frame().rename(columns = {"review":"lower_case"}))
spl_chars_df = merge_two_dataframes(lower_case_df, lower_case_df['lower_case'].apply(lambda a: remove_spl_characters(a)).to_frame().rename(columns = {"lower_case":"string_after_removing_spl_chars"}))
stop_words_rm_df = merge_two_dataframes(spl_chars_df, spl_chars_df['string_after_removing_spl_chars'].apply(lambda a:remove_stop_words(a.split())).to_frame().rename(columns = {"string_after_removing_spl_chars":"stop_words_removal"}))
#5 frequent words
frequent_words = list(pd.Series(' '.join(stop_words_rm_df['string_after_removing_spl_chars']).split()).value_counts()[:5].index)
#5 rare words
rare_words = list(pd.Series(' '.join(stop_words_rm_df['string_after_removing_spl_chars']).split()).value_counts()[-5:].index)
frequent_words_df = merge_two_dataframes(stop_words_rm_df, stop_words_rm_df['stop_words_removal'].apply(lambda a:remove_frequent_words(a.split())).to_frame().rename(columns = {"stop_words_removal":"frequent_words"}))
rare_words_df = merge_two_dataframes(frequent_words_df, stop_words_rm_df['stop_words_removal'].apply(lambda a:remove_frequent_words(a.split())).to_frame().rename(columns = {"stop_words_removal":"rare_words"}))
spell_corrector_df = merge_two_dataframes(rare_words_df, rare_words_df['rare_words'].apply(lambda a:remove_frequent_words(a.split())).to_frame().rename(columns = {"rare_words":"spelling_check"}))
required_df = spell_corrector_df[["Names","review","spelling_check"]].rename(columns = {"spelling_check":"cleaned_data"})
required_df.head()

,Names,review,cleaned_data
0,MihaVrhunc,"Every once in a while a movie comes, that trul...",every movie comes truly makes impact joaquins ...
1,lesterarnoldpinto,This is a movie that only those who have felt ...,movie felt alone isolated truly relate underst...
2,Aman_Goyal,"Truly a masterpiece, The Best Hollywood film o...",truly masterpiece best hollywood film 2019 one...
3,logical_guy,Joaquin Phoenix gives a tour de force performa...,joaquin phoenix gives tour de force performanc...
4,kdagoulis26,Most of the time movies are anticipated like t...,time movies anticipated like end falling short...


In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
#1.1
import nltk
from nltk.util import ngrams
from collections import Counter
import re
result=list()
for data in required_df['cleaned_data']:
  n_grams = ngrams(nltk.word_tokenize(data),3)
  for grams in n_grams:
      result.append(' '.join(grams))
Counter(result)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: generator 'ngrams' raised StopIteration
  if __name__ == '__main__':


Counter({'every movie comes': 1,
         'movie comes truly': 1,
         'comes truly makes': 1,
         'truly makes impact': 1,
         'makes impact joaquins': 1,
         'impact joaquins performance': 1,
         'joaquins performance scenography': 1,
         'performance scenography brilliance': 1,
         'scenography brilliance grotesque': 1,
         'brilliance grotesque haunting': 1,
         'grotesque haunting cringy': 1,
         'haunting cringy hard': 1,
         'cringy hard watch': 1,
         'hard watch times': 1,
         'watch times mesmerizing': 1,
         'times mesmerizing wont': 1,
         'mesmerizing wont blink': 1,
         'wont blink eye': 1,
         'blink eye watching': 1,
         'eye watching tragic': 1,
         'watching tragic seriously': 1,
         'tragic seriously funny': 1,
         'seriously funny moments': 1,
         'funny moments emotional': 1,
         'moments emotional rollercoaster': 1,
         'emotional rollercoaster so

In [ ]:
import warnings
warnings.filterwarnings('ignore')


In [ ]:
#1.2
for data in required_df['cleaned_data']:
  n_grams = ngrams(nltk.word_tokenize(data),2)
  bigrams = [' '.join(val) for val in n_grams]
counts=Counter(bigrams)

In [ ]:
keys = [i[0] for i in counts.items()]
tokens = [i[0].split(" ") for i in counts.items()]
words = [i[0] for i in tokens]
spl_chr=list(filter(None,words))

In [ ]:
for i in tokens:
  count=0
  for spl in spl_chr:
    if i[0]==spl:
      count+=1
  count_values.append(count)

In [ ]:
for temp in range(len(list(counts.values()))):
  print(keys[temp],"->",round(list(counts.values())[temp]/ count_values[temp],2))


want start -> 1.0
start review -> 1.0
review saying -> 1.0
saying avid -> 1.0
avid marvel -> 1.0
marvel movie -> 0.25
movie fan -> 1.0
fan many -> 1.0
many years -> 1.0
years embarrassed -> 1.0
embarrassed dc -> 1.0
dc attempts -> 0.33
attempts catch -> 1.0
catch game -> 1.0
game however -> 1.0
however film -> 1.0
film flipped -> 0.25
flipped entire -> 1.0
entire script -> 1.0
script marvel -> 1.0
marvel found -> 0.25
found good -> 1.0
good formula -> 1.0
formula directors -> 1.0
directors russo -> 1.0
russo brothers -> 1.0
brothers writers -> 1.0
writers taika -> 1.0
taika waititi -> 1.0
waititi combined -> 1.0
combined elements -> 1.0
elements action -> 1.0
action comedy -> 1.0
comedy story -> 1.0
story produce -> 1.0
produce enjoyable -> 1.0
enjoyable films -> 1.0
films todd -> 0.5
todd phillips -> 1.0
phillips take -> 1.0
take joker -> 1.0
joker everything -> 1.0
everything right -> 1.0
right dc -> 1.0
dc films -> 0.33
films missing -> 0.5
missing film -> 1.0
film meant -> 0.25
mea

In [ ]:
!pip install textacy

In [ ]:
#1.3
import textacy
#create a list of sublists
nouns_list = [[j for j in textacy.make_spacy_doc(i,lang='en_core_web_sm').noun_chunks] for i in  required_df["cleaned_data"] ]
final_list = list(itertools.chain.from_iterable(nouns_list))

In [ ]:
sentences = ""
for i in required_df["cleaned_data"]:
  sentences = sentences+' ; '+i

In [ ]:
import itertools
from itertools import chain

In [ ]:
#compute noun frequency
noun_frequency = list()
for noun in nouns_list:
  for token in noun:
    list_of_values = [data.count(str(token))/sentences.count(str(token)) for data in required_df["cleaned_data"]]
    noun_frequency.append(list_of_values)

In [ ]:

result_df=pd.DataFrame(noun_frequency).T
result_df.columns = final_list

In [ ]:
result_df.head(5)

,"(every, movie)","(impact, joaquins, performance, scenography, brilliance)","(cringy, hard, watch, times)",(eye),"(tragic, seriously, funny, moments, emotional, rollercoaster)","(single, best, character, development)",(movie),"(isolated, truly, relate, understand)","(character, lot, people)",(movie),(violence),"(truly, movie)",(us),"(better, person)",(everyone),(respect),"(belong, world)","(truly, masterpiece, best, hollywood, film)","(one, best, films, decade)","(truly, best, film)","(comic, book, chillingly, realistically, real, ife, remarkable, direction, cinematography, music)","(acting, people)","(disturbing, violent, necessity, message, society)","(underappreciatedunrecognizedbullied, people)","(class, difference, corruption, rich, talented, rule, others)",(s),"(multiple, jokers, living, society)","(much, bitter, way)",(film),(people),(people),(wake),"(joaquin, phoenix)","(tour, de, force, performance, fearless, stunning, emotional, depth, physicality)","(impossible, talk)","(heath, ledgers)","(performance, dark, knight)","(definitive, liveaction, portrayal, joker)","(talk, fact)",(everyone),...,"(best, dc, universe, movie, joker)","(acting, film)","(movie, audience)",(joker),"(thomas, wayne, batmans, father)","(uncaring, movie)",(viewer),"(first, minutes, necessary, message, movie, powerful, contemporary, issues)","(lot, story, line)","(good, pace)","(movie, goodthe, cinematographythe, actingwriting)","(definitely, best, movie)","(start, review)","(avid, marvel, movie, fan)",(dc),"(catch, game)",(film),"(good, formula, directors)","(brothers, writers)",(taika),"(waititi, combined, elements, action, comedy, story)","(enjoyable, films)","(todd, phillips)",(joker),(everything),"(dc, films)",(film),"(nice, warm, feeling)","(walking, marvel, film, challenge, perception, society, issues)","(achieves, waves)",(dc),(something),(marvel),(world),"(joaquin, phoenix, course)","(exceptional, role)",(role),"(real, understanding, entails)",(film),(anything)
0,1.0,1.0,1.0,0.5,1.0,1.0,0.010811,0.0,0.0,0.010811,0.0,0.0,0.013158,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.012092,0.0,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.016216,1.0,1.0,0.016216,0.1,1.0,0.013158,1.0,0.2,0.5,1.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.006046,0.0,0.0,0.00,0.033333,0.033333,0.000000,0.000000,0.0,0.0,0.00,0.0,0.0,0.0,0.2,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.00,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.013158,0.0,0.0,0.0,0.0,0.5,0.333333,0.5,0.5,0.5,0.5,0.5,0.5,0.019952,0.5,0.5,0.08,0.133333,0.133333,0.333333,0.000000,0.0,0.0,0.00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.009346,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.08,0.0,0.0,0.0,0.0,0.0,0.0,0.009346,0.0,0.0,0.08,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.08,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.2,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.011487,0.0,0.0,0.00,0.000000,0.000000,0.000000,0.030303,1.0,1.0,0.25,1.0,1.0,1.0,0.2,...,0.0,0.0,0.0,0.009346,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.009346,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.010811,0.0,0.0,0.010811,0.1,0.0,0.013158,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.010883,0.0,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.018692,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.018692,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0



# **Question 2: Undersand TF-IDF and Document representation**


(40 points). Starting from the documents (all the reviews, or abstracts, or tweets) collected for assignment two, write a python program:

(1) To build the **documents-terms weights (tf*idf) matrix bold text**.

(2) To rank the documents with respect to query (design a query by yourself, for example, "An Outstanding movie with a haunting performance and best character development") by using **cosine similarity**.

In [ ]:
required_df.head(5)

,Names,review,cleaned_data
0,MihaVrhunc,"Every once in a while a movie comes, that trul...",every movie comes truly makes impact joaquins ...
1,lesterarnoldpinto,This is a movie that only those who have felt ...,movie felt alone isolated truly relate underst...
2,Aman_Goyal,"Truly a masterpiece, The Best Hollywood film o...",truly masterpiece best hollywood film 2019 one...
3,logical_guy,Joaquin Phoenix gives a tour de force performa...,joaquin phoenix gives tour de force performanc...
4,kdagoulis26,Most of the time movies are anticipated like t...,time movies anticipated like end falling short...


In [ ]:
#2.1
#fetch the ist of all the words
res = list()
for i in required_df["cleaned_data"]:
  res.append(i.split(" "))
words = list(set([j for i in res for j in i if len(j)>0]))
result_df = pd.DataFrame(words,columns=["Words"])

In [ ]:
import math
def find_value(x,sentence):
  value = sentence.count(x);
  if(value != 0):
    return (sentence.count(x)/len(sentence.split(" ")))*(math.log(len(sentence.split(" "))/sentence.count(x), 10))
  else:
    return 0

In [ ]:
result_df
i=1

for data in required_df["cleaned_data"]:
  result_df["review"+str(i)] = result_df["Words"].apply(lambda x: find_value(x,str(data)))
  i = i+1

In [ ]:
result_df

,Words,review1,review2,review3,review4,review5,review6,review7,review8,review9,review10,review11,review12,review13,review14,review15,review16,review17,review18,review19,review20,review21,review22,review23,review24,review25,review26,review27,review28,review29,review30,review31,review32,review33,review34,review35,review36,review37,review38,review39,...,review61,review62,review63,review64,review65,review66,review67,review68,review69,review70,review71,review72,review73,review74,review75,review76,review77,review78,review79,review80,review81,review82,review83,review84,review85,review86,review87,review88,review89,review90,review91,review92,review93,review94,review95,review96,review97,review98,review99,review100
0,says,0.000000,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.000000,0.0,0,0.0,0.0,0.0,0.0,0.000000,0.000000,0,0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.012228,0,0.000000,0.0,0,0.000000,...,0,0.0,0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,regardless,0.000000,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.000000,0.0,0,0.0,0.0,0.0,0.0,0.000000,0.000000,0,0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0,0.000000,0.0,0,0.000000,...,0,0.0,0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.006819,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,harmony,0.000000,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.000000,0.0,0,0.0,0.0,0.0,0.0,0.000000,0.000000,0,0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.011505,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0,0.000000,0.0,0,0.000000,...,0,0.0,0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,despite,0.000000,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.015329,0.0,0,0.0,0.0,0.0,0.0,0.000000,0.000000,0,0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0,0.000000,0.0,0,0.028222,...,0,0.0,0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,accomplishes,0.000000,0.0,0.0,0.035026,0.000000,0.00000,0.0,0.000000,0.0,0,0.0,0.0,0.0,0.0,0.000000,0.000000,0,0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0,0.000000,0.0,0,0.000000,...,0,0.0,0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1691,setting,0.000000,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.000000,0.0,0,0.0,0.0,0.0,0.0,0.000000,0.000000,0,0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.081866,0.000000,0.0,0.0,0.000000,0,0.000000,0.0,0,0.000000,...,0,0.0,0,0.0,0.0,0.0,0.000000,0.0,0.0,0.027891,0.0,0.0,0.0,0.0,0.0,0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1692,short,0.000000,0.0,0.0,0.000000,0.059206,0.00000,0.0,0.000000,0.0,0,0.0,0.0,0.0,0.0,0.000000,0.000000,0,0,0.0,0.0,0.040797,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.006926,0,0.011842,0.0,0,0.000000,...,0,0.0,0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.00

In [ ]:
#2.2


# **Question 3: Create your own training and evaluation data for sentiment analysis**


(15 points). **You dodn't need to write program for this question!** Read each review (abstract or tweet) you collected in detail, and annotate each review with a sentiment (positive, negative, or neutral). Save the annotated dataset into a csv file with three columns (first column: document_id, clean_text, sentiment), upload the csv file to GitHub and submit the file link blew. This datset will be used for assignment four: sentiment analysis and text classification.

In [169]:
required_df.to_csv("Sentiment Analysis and Text Classification.csv")

file information


https://github.com/akhiltabjula/ComputationalMethodsForInformationSystems/blob/master/Sentiment%20Analysis%20and%20Text%20Classification.csv